In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import fast_dsift, no_op
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images

from menpofit.atm import (
    ATMBuilder, PatchATMBuilder, LinearATMBuilder, LinearPatchATMBuilder, PartsATMBuilder,
    LKATMFitter, 
    FC, IC)
from menpofit.transform import DifferentiableAlignmentAffine
from menpofit.visualize import visualize_fitting_result

## Load data

In [ ]:
images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=100):
    labeller(i, 'PTS', ibug_face_66)
    i.crop_to_landmarks_proportion(0.5, group='PTS')
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    images.append(i)

In [ ]:
template = images[0]
shapes = [i.landmarks['ibug_face_66'].lms for i in images]
test_images = images[:5]

In [ ]:
visualize_images(template)

In [ ]:
visualize_images(test_images)

## Build Active Template Models

In [ ]:
features = fast_dsift
diagonal = 100
scales = (1, .5)
scale_shapes = False
scale_features = True
max_shape_components= 25

patch_shape = (17, 17)

normalize_parts = no_op

#### ATM

In [ ]:
atm_builder = ATMBuilder(
    features=features,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components)

atm = atm_builder.build(
    shapes,
    template, 
    group='ibug_face_66', 
    verbose=True)

#### Patch based ATM

In [ ]:
patch_atm_builder = PatchATMBuilder(
    patch_shape=patch_shape,
    features=features,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components)

patch_atm = patch_atm_builder.build(
    shapes,
    template, 
    group='ibug_face_66', 
    verbose=True)

#### Linear ATM

In [ ]:
linear_atm_builder = LinearATMBuilder(
    features=features,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components)

linear_atm = linear_atm_builder.build(
    shapes,
    template, 
    group='ibug_face_66', 
    verbose=True)

#### Linear Patch based ATM

In [ ]:
linear_patch_atm_builder = LinearPatchATMBuilder(
    patch_shape=patch_shape,
    features=features,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components)

linear_patch_atm = linear_patch_atm_builder.build(
    shapes,
    template, 
    group='ibug_face_66', 
    verbose=True)

#### Parts based ATM

In [ ]:
parts_atm_builder = PartsATMBuilder(
    patch_shape=patch_shape, 
    features=features,
    normalize_parts=normalize_parts,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components)

parts_atm = parts_atm_builder.build(
    shapes,
    template, 
    group='ibug_face_66', 
    verbose=True)

## Fit Active Template Models

In [ ]:
algorithm_cls = IC
sampling_step = 8

#### ATM Fitter

In [ ]:
atm_fitter = LKATMFitter(
    atm, 
    algorithm_cls=algorithm_cls, 
    n_shape=[3, 12],
    sampling=sampling_step)

#### Patch based ATM Fitter

In [ ]:
patch_atm_fitter = LKATMFitter(
    patch_atm, 
    algorithm_cls=algorithm_cls, 
    n_shape=[3, 12],
    sampling=sampling_step)

#### Linear ATM Fitter

In [ ]:
linear_atm_fitter = LKATMFitter(
    linear_atm, 
    algorithm_cls=algorithm_cls, 
    n_shape=[3, 12],
    sampling=sampling_step)

#### Linear Patch ATM Fitter

In [ ]:
linear_patch_atm_fitter = LKATMFitter(
    linear_patch_atm, 
    algorithm_cls=algorithm_cls, 
    n_shape=[3, 12],
    sampling=sampling_step)

#### Parts based ATM Fitter

In [ ]:
sampling_mask = parts_mask = np.require(np.zeros(patch_shape, dtype=np.bool))
sampling_mask[2::3, 2::3] = True

imshow(sampling_mask)

In [ ]:
parts_atm_fitter = LKATMFitter(
    parts_atm, 
    algorithm_cls=algorithm_cls, 
    n_shape=[3, 12],
    sampling=sampling_mask)

#### Fitting loop

In [ ]:
fitters = [atm_fitter,
           patch_atm_fitter,
           linear_atm_fitter,
           linear_patch_atm_fitter,
           parts_atm_fitter]

fitters_results = []

for j, i in enumerate(test_images):
    
    gt_s = i.landmarks['ibug_face_66'].lms
    s = fitters[0].noisy_shape_from_shape(gt_s, noise_std=0.04)

    for fitter in fitters:
        
        fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=20) 
        fr.downscale = 0.5
        fitters_results.append(fr)
    
        print 'Image: ', j
        print fr

## Visualize Results

In [ ]:
visualize_fitting_result(fitters_results)